# Project 05: Ensemble Machine Learning - Wine Dataset
**Author:**  James Pinkston  
**Date:**  November 23, 2025  
**Objective:**  P5:  This project will evaluate model performance by exploring ensemble models by using the Wine Quality Dataset located <a href="https://archive.ics.uci.edu/ml/datasets/Wine+Quality" target="_blank">here</a>.

## Introduction to Ensemble Models
Ensemble models combine the outputs of multiple models to improve predictive performance. Common types of ensemble models include:

- Boosted Decision Trees – Models train sequentially, with each new tree correcting the errors of the previous one.
- Random Forest – Multiple decision trees train in parallel, each on a random subset of the data, and their predictions are averaged.
- Voting Classifier (Heterogeneous Models) – Combines different types of models (e.g., Decision Tree, SVM, and Neural Network) by taking the majority vote or average prediction.
- Cross Validation – Divides data into multiple folds to improve the reliability of performance estimates.

## Performance Metrics
We will evaluate model performance using the following metrics:

- Accuracy –  The proportion of all predictions that are correct.
- Precision – Proportion of positive predictions that are truly positive.
- Recall – Proportion of actual positives that are correctly predicted.
- F1 Score – Harmonic mean of precision and recall, balancing both.

These metrics are especially helpful when working with multiple classes (e.g., low, medium, high), not just binary yes/no predictions.

Good models have:

- High Test Accuracy – the model predicts well on new, unseen (e.g., test) data.
- High Test F1 Score – especially useful  if some classes (categories) have fewer examples than others.
- Small Gap between Train and Test accuracy – shows the model is generalizing well (not overfitting or underfitting).
- Small Gap between Train and Test F1 score – shows the model is generalizing well (not overfitting or underfitting).